In [1]:
import pandas as pd
import time

In [2]:
with open('onegin.txt', 'r') as f:
    d = f.read()

In [3]:
oneg = d.split('\n')

In [4]:
a = []
b = []
count = 0
for i in oneg:
    if i[:2] == '\t\t' and i[:4] != '\t\t……':
        a.append(i[2:])
    count += 1
    if count % 4 == 0 and a != []:
        b.append(' '.join(a))
        a = []

In [5]:
text = [x.lower() for x in b]

In [6]:
text[:10]

['he мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить',
 'залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной,',
 'высоких дум и простоты; но так и быть – рукой пристрастной прими собранье пёстрых глав, полусмешных, полупечальных,',
 'простонародных, идеальных, небрежный плод моих забав, бессонниц, лёгких вдохновений, незрелых и увядших лет,',
 'ума холодных наблюдений и сердца горестных замет.',
 '«мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог.',
 'его пример другим наука; но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь!',
 'какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство,',
 'вздыхать и думать про себя: когда же чёрт возьмёт тебя!»',
 'так думал молодой повеса, летя в пыли на почтовых,']

In [7]:
CHARS = set('абвгдеёжзийклмнопрстуфхцчшщыьъэюя.,?! ')

In [8]:
len(CHARS)

38

In [9]:
text2 = []
for i in text:
    line = ''
    for j in i:
        if j in CHARS:
            line = line+j 
    text2.append(line)

In [10]:
text2[:10]

[' мысля гордый свет забавить, вниманье дружбы возлюбя, хотел бы я тебе представить',
 'залог достойнее тебя, достойнее души прекрасной, святой исполненной мечты, поэзии живой и ясной,',
 'высоких дум и простоты но так и быть  рукой пристрастной прими собранье пёстрых глав, полусмешных, полупечальных,',
 'простонародных, идеальных, небрежный плод моих забав, бессонниц, лёгких вдохновений, незрелых и увядших лет,',
 'ума холодных наблюдений и сердца горестных замет.',
 'мой дядя самых честных правил, когда не в шутку занемог, он уважать себя заставил и лучше выдумать не мог.',
 'его пример другим наука но, боже мой, какая скука с больным сидеть и день и ночь, не отходя ни шагу прочь!',
 'какое низкое коварство полуживого забавлять, ему подушки поправлять, печально подносить лекарство,',
 'вздыхать и думать про себя когда же чёрт возьмёт тебя!',
 'так думал молодой повеса, летя в пыли на почтовых,']

In [11]:
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]

In [12]:
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [13]:
CHAR_TO_INDEX

{'none': 0,
 'к': 1,
 'ь': 2,
 'э': 3,
 'ж': 4,
 'ё': 5,
 'б': 6,
 'р': 7,
 '?': 8,
 'м': 9,
 ',': 10,
 'в': 11,
 'г': 12,
 'л': 13,
 'щ': 14,
 'з': 15,
 'ф': 16,
 'и': 17,
 'ъ': 18,
 'ы': 19,
 '.': 20,
 'п': 21,
 'с': 22,
 'у': 23,
 'х': 24,
 ' ': 25,
 'ц': 26,
 'д': 27,
 'ю': 28,
 'ч': 29,
 'а': 30,
 'й': 31,
 'о': 32,
 'е': 33,
 'н': 34,
 'я': 35,
 '!': 36,
 'ш': 37,
 'т': 38}

In [14]:
import torch

In [15]:
MAX_LEN = 50

In [16]:
X = torch.zeros((len(text2), MAX_LEN), dtype=int)

In [17]:
for i in range(len(text2)):
    for j, w in enumerate(text2[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [18]:
X[0:1]

tensor([[25,  9, 19, 22, 13, 35, 25, 12, 32,  7, 27, 19, 31, 25, 22, 11, 33, 38,
         25, 15, 30,  6, 30, 11, 17, 38,  2, 10, 25, 11, 34, 17,  9, 30, 34,  2,
         33, 25, 27,  7, 23,  4,  6, 19, 25, 11, 32, 15, 13, 28]])

In [19]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.gru = torch.nn.RNN(28, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [20]:
model = Network()

In [21]:
model.forward(X[0:1])[0].shape

torch.Size([1, 50, 39])

In [22]:
dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [23]:
model = model.to(dev)

In [24]:
def generate_sentence():
    sentence = ['м', 'о', 'й', ' ', 'д', 'я', 'д', 'я']
    state = None
    for i in range(MAX_LEN):
        X = torch.Tensor([[CHAR_TO_INDEX[sentence[i]]]]).type(torch.long).to(dev)
        if i == 0:
            result, state = model.forward(X)
        else:
            result, state = model.forward_state(X, state)
        prediction = result[0, -1, :]
        index_of_prediction = prediction.argmax()
        if i >= len(sentence) - 1:
            if index_of_prediction == 0:
                break
        sentence.append(INDEX_TO_CHAR[index_of_prediction])
    print(''.join(sentence))

In [25]:
generate_sentence()

мой дядя иьжэц лъшы!ьнъ.ъъцяъсь ьёжлёбъяъфб,дсььъ.с!эйп!ё 


In [26]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.005, momentum=0.7)

In [27]:
for ep in range(10000):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        X_batch = X_batch.to(dev)
        Y_batch = Y_batch.to(dev)

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    
    if ep % 100 == 0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        generate_sentence()


Epoch 0. Time: 0.030, Train loss: 3.669
мой дядя иьжэц лъшы!ьнъ.ъъцяъсь ьёжлёбъяъфб,дсььъ.с!эйп!ё 
Epoch 100. Time: 0.028, Train loss: 2.676
мой дядя   но   ноое ноое   но   ноое ноое   но   ноое ноо
Epoch 200. Time: 0.028, Train loss: 2.522
мой дядя   по   нннонпннее        нонннннне е      н пнннн
Epoch 300. Time: 0.027, Train loss: 2.441
мой дядя   по   ннпонпнпее  о и   пннп пнпоеа пое    нон н
Epoch 400. Time: 0.028, Train loss: 2.388
мой дядя   пе   ннпоспппее  тооон пво    воонппппот о ооон
Epoch 500. Time: 0.012, Train loss: 2.348
мой дядяо  пе    нпосппппеонтоооосно     то нппппо паоооон
Epoch 600. Time: 0.028, Train loss: 2.317
мой дядяо  пе    нпосппппеонтоооотны      ы ппппппепоооооо
Epoch 700. Time: 0.013, Train loss: 2.290
мой дядяо  пе    нпррппппеоиаоооотн       ывппппппе оооооо
Epoch 800. Time: 0.013, Train loss: 2.267
мой дядяо  пе    нпррппппеоиаоооотн       ывппппппе оооооо
Epoch 900. Time: 0.023, Train loss: 2.247
мой дядяо  пе    нвррпвппеоеаоооотн       ыввпппппл

In [33]:
class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.word_embeddings = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.gru = torch.nn.GRU(28, 128, batch_first=True)
        self.hidden2tag = torch.nn.Linear(128, len(INDEX_TO_CHAR))

    def forward(self, sentences):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

    def forward_state(self, sentences, state):
        embeds = self.word_embeddings(sentences)
        gru_out, state = self.gru(embeds, state)
        tag_space = self.hidden2tag(gru_out.reshape(-1, 128))
        return tag_space.reshape(sentences.shape[0], sentences.shape[1], -1), state

In [34]:
model = Network()

In [35]:
model = model.to(dev)

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [41]:
def train_net(epocha):
    for ep in range(epocha):
        start = time.time()
        train_loss = 0.
        train_passed = 0

        for i in range(int(len(X) / 100)):
            batch = X[i * 100:(i + 1) * 100]
            X_batch = batch[:, :-1]
            Y_batch = batch[:, 1:].flatten()
            X_batch = X_batch.to(dev)
            Y_batch = Y_batch.to(dev)

            optimizer.zero_grad()
            answers, _ = model.forward(X_batch)
            answers = answers.view(-1, len(INDEX_TO_CHAR))
            loss = criterion(answers, Y_batch)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()
            train_passed += 1


        if ep % 100 == 0:
            print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
            generate_sentence()

In [48]:
optimizer = torch.optim.Adam(model.parameters(), lr=.001)

In [49]:
train_net(5000)

Epoch 0. Time: 0.035, Train loss: 0.080
мой дядяои дядя н иетн сасх рие л  доктсопнллооиюриыы нд ы
Epoch 100. Time: 0.033, Train loss: 0.068
мой дядяеи дядя с кедятстмннилеьвиыо   нетннкзпо  ыоооствс
Epoch 200. Time: 0.034, Train loss: 0.068
мой дядяеи дядя с кедятсамлрилея июякаттн    ,еаыдпл  здна
Epoch 300. Time: 0.032, Train loss: 0.067
мой дядяеи дядя с кедятсамлрилея июокаттн ббр ьаиолеан ж н
Epoch 400. Time: 0.033, Train loss: 0.066
мой дядяеи дядя с кедятсамлрилея июякаттн б  ,еаыдеэс злна
Epoch 500. Time: 0.033, Train loss: 0.066
мой дядяеи дядя с кедятсамлрилея июякаттн     еаыдплмм,л н
Epoch 600. Time: 0.033, Train loss: 0.103
мой дядяои дядя т неся соие л иобм суннкыууудоыахд  екбл и
Epoch 700. Time: 0.033, Train loss: 0.068
мой дядяои дядя т неси сома ллив албео аоньоннкл а .аыояе 
Epoch 800. Time: 0.033, Train loss: 0.067
мой дядяеи дядя с кедятстжлрисеьвеюоккймат  ,о   улп квббм
Epoch 900. Time: 0.033, Train loss: 0.066
мой дядяеи дядя с кедятстжлрисеьвеюоккй,ат  ,о   е 

In [ ]:
# лосс больше не падает - сеть генерирует отдельные слова. Попробовал ванильную RNN и GRU. Результат сильно не улушился.